(seb:exercise)=
### Surface Energy Balance model

The aim of this exercise is to understand how we can solve simple optimisation problems. To do this, we will develop a simple surface energy balance model (SEB). Since energy can neither be produced nor destroyed, the sum of the energy fluxes at the Earth's surface must be zero. If the static quantities such as roughness length, albedo, stability etc. are known and quantities such as temperature and humidity are measured, the balance of the energy fluxes at the surface is only a function of the surface temperature.

For simplicity, we parameterise the turbulent fluxes with a bulk approach and neglect the soil heat flux. However, at the end of this exercise, we will consider the soil heat flux by coupling the heat conduction equation to the energy balance model.

**Task 1**: Develop a simple SEB model. The turbulent flows are to be parameterised using a simple bulk approach. Write a function that takes the following arguments: surface temperature, air temperature, relative humidity, albedo, global radiation, atmospheric pressure, air density, wind speed, altitude measured and roughness length. The function should return the short-wave radiation balance and the two turbulent energy fluxes.

In [14]:
import math
import numpy as np
from scipy.optimize import minimize, minimize_scalar
import matplotlib.pyplot as plt

%matplotlib inline


def EB_fluxes(T_0,T_a,f,albedo,G,p,rho,U_L,z,z_0):
    """ This function calculates the energy fluxes from the following quantities:
    
    Input: 
    T_0       : Surface temperature, which is optimized [K]
    f         : Relative humdity as fraction, e.g. 0.7 [-]
    albedo    : Snow albedo [-]
    G         : Shortwave radiation [W m^-2]
    p         : Air pressure [hPa]
    rho       : Air denisty [kg m^-3]
    z         : Measurement height [m]
    z_0       : Roughness length [m]
    
    """
    
        # Correction factor for incoming longwave radiation
    eps_cs = 0.23 + 0.433 * np.power(100*(f*E_sat(T_a))/T_a,1.0/8.0)
    
    # Select the appropriate latent heat constant
    L = 2.83e6 # latent heat for sublimation

    # Calculate turbulent fluxes
    print(f'H_0 = {rho} * {c_p} * {Cs_t} * {U_L} * ({T_0} - {T_a})')
    H_0 = rho * c_p * Cs_t * U_L * (T_0 - T_a)
    #E_0 = rho * L*0.622/p * Cs_q * U_L * (E_sat(T_0) - f*E_sat(T_0))
    E_0 = rho * L*0.622/p * Cs_q * U_L * E_sat(T_0)*(1 - f)
    
    # Calculate radiation budget
    L_d = eps_cs * sigma * T_a**4
    L_u = 1.0 * sigma * T_0**4
    Q_0 = (1-albedo) * G

    return (Q_0, L_d, L_u, H_0, E_0)

def E_sat(T):
    """ Saturation water vapor equation """
    Ew = 6.112 * np.exp((17.67*(T-273.16)) / ((T-29.66)))
    print(Ew)
    return Ew


In [15]:
# Test the SEB function
# Define necessary variables and parameters
T_0 = 283.0   # Surface temperature
T_a = 280.0   # Air temperature 
f = 0.7       # Relative humidity
albedo = 0.3  # albedo
G = 700.0     # Incoming shortwave radiation
rho = 1.1     # Air density
U = 2.0       # Wind velocity
z =  2.0      # Measurement height
z0 = 1e-3     # Roughness length
p = 1013      # Pressure


# Some constants
c_p = 1004            # specific heat [J kg^-1 K^-1]
kappa = 0.41          # Von Karman constant [-]
sigma = 5.67e-8       # Stefan-Bolzmann constant
    
# Bulk coefficients 
Cs_t = kappa**2 / (np.log(z/z0)**2)
Cs_q = Cs_t

print(Cs_t)

# Run the function
Q_0, L_d, L_u, H_0, E_0 = EB_fluxes(T_0,T_a,f,albedo,G,p,rho,U,z,z0)

# Print results
print('Surface temperature: {:.2f}'.format(T_0))
print('Global radiation: {:.2f}'.format(Q_0))
print('Longwave down: {:.2f}'.format(L_d))
print('Longwave up: {:.2f}'.format(L_u))
print('Surface heat flux: {:.2f}'.format(H_0))
print('Latent heat flux: {:.2f}'.format(E_0))

0.002909627512974121
9.905088858733954
H_0 = 1.1 * 1004 * 0.002909627512974121 * 2.0 * (283.0 - 280.0)
12.14085232018689
Surface temperature: 283.00
Global radiation: 490.00
Longwave down: 249.17
Longwave up: 363.69
Surface heat flux: 19.28
Latent heat flux: 40.51


**Task 2**: Now we need to optimize for the surface temperature. Therefore, we need to write a so-called optimization function. In our case the sum of all fluxes should be zero. The SEB depends on the surface temperature. So we have to find the surface temperature which fulfills the condition $SEB(T_0)=Q_0+L_d-L_u-H_0-E_0=0$. 

In [16]:
def optim_T0(x,T_a,f,albedo,G,p,rho,U_L,z,z0):
    """ Optimization function for surface temperature:
    
    Input: 
    T_0       : Surface temperature, which is optimized [K]
    f         : Relative humdity as fraction, e.g. 0.7 [-]
    albedo    : Snow albedo [-]
    G         : Shortwave radiation [W m^-2]
    p         : Air pressure [hPa]
    rho       : Air denisty [kg m^-3]
    z         : Measurement height [m]
    z_0       : Roughness length [m]
    
    """
    
    Q_0, L_d, L_u, H_0, E_0 = EB_fluxes(x,T_a,f,albedo,G,p,rho,U_L,z,z0)
    print((Q_0,L_d,L_u,H_0,E_0))
    
    # Get residual for optimization
    res = np.abs(Q_0+L_d-L_u-H_0-E_0)
    print(res)
    # return the residuals
    return res

We use the **minimize function** from the scipy module to find the temperature values. 

In [17]:
# Test the SEB function
# Define necessary variables and parameters
T_0 = 283.0   # Surface temperature
T_a = 280.0   # Air temperature 
f = 0.7       # Relative humidity
albedo = 0.3  # albedo
G = 700.0     # Incoming shortwave radiation
rho = 1.1     # Air density
U = 2.0       # Wind velocity
z =  2.0      # Measurement height
z0 = 1e-3     # Roughness length
p = 1013      # Pressure

test_args = (T_a,f,albedo,G,p,rho,U,z,z0)
print(f'Test args: {test_args}')
# Run the function
res = minimize(optim_T0,x0=T_0,args=test_args,bounds=((None,400),), \
                         method='L-BFGS-B',options={'eps':1e-8})

res

Test args: (280.0, 0.7, 0.3, 700.0, 1013, 1.1, 2.0, 2.0, 0.001)
9.905088858733954
H_0 = 1.1 * 1004 * 0.002909627512974121 * 2.0 * ([283.] - 280.0)
[12.14085232]
(489.99999999999994, 249.17331207257877, array([363.68785712]), array([19.28035575]), array([40.5133244]))
[315.6917748]
9.905088858733954
H_0 = 1.1 * 1004 * 0.002909627512974121 * 2.0 * ([283.00000001] - 280.0)
[12.14085233]
(489.99999999999994, 249.17331207257877, array([363.68785717]), array([19.28035582]), array([40.51332443]))
[315.69177466]
9.905088858733954
H_0 = 1.1 * 1004 * 0.002909627512974121 * 2.0 * ([284.] - 280.0)
[12.97924419]
(489.99999999999994, 249.17331207257877, array([368.85563205]), array([25.707141]), array([43.31098973]))
[301.29954929]
9.905088858733954
H_0 = 1.1 * 1004 * 0.002909627512974121 * 2.0 * ([284.00000001] - 280.0)
[12.9792442]
(489.99999999999994, 249.17331207257877, array([368.8556321]), array([25.70714107]), array([43.31098976]))
[301.29954914]
9.905088858733954
H_0 = 1.1 * 1004 * 0.0029096

      fun: array([1.00894113e-07])
 hess_inv: <1x1 LbfgsInvHessProduct with dtype=float64>
      jac: array([0.04041848])
  message: 'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
     nfev: 122
      nit: 8
     njev: 61
   status: 0
  success: True
        x: array([301.68719865])

The temperature value is stored in the x value of the result dictionary

In [18]:
# Assign optimization result to variable T_0
T_0 = res.x[0]

# Run the function
Q_0, L_d, L_u, H_0, E_0 = EB_fluxes(T_0,T_a,f,albedo,G,p,rho,U,z,z0)

# Print results
print('Surface temperature: {:.2f}'.format(T_0))
print('Global radiation: {:.2f}'.format(Q_0))
print('Longwave down: {:.2f}'.format(L_d))
print('Longwave up: {:.2f}'.format(L_u))
print('Surface heat flux: {:.2f}'.format(H_0))
print('Latent heat flux: {:.2f}'.format(E_0))

9.905088858733954
H_0 = 1.1 * 1004 * 0.002909627512974121 * 2.0 * (301.6871986496958 - 280.0)
38.98932442022194
Surface temperature: 301.69
Global radiation: 490.00
Longwave down: 249.17
Longwave up: 469.69
Surface heat flux: 139.38
Latent heat flux: 130.11


In [13]:
T_0

301.6871986496958